# Турнир «Камень-Ножницы-Бумага» с использованием 13 агентов

In [2]:

from kaggle_environments import make
import random
import pandas as pd
    

Matplotlib created a temporary cache directory at /var/folders/n5/twq2syy550527g4q5jk2rg7h0000gn/T/matplotlib-0bmpwawf because the default path (/Users/pavelminsky/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


termcolor not installed, skipping dependency
No pygame installed, ignoring import


## Определение агентов

In [4]:

# Базовые агенты
def rock_agent(observation, configuration):
    return 0

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

def paper_agent(observation, configuration):
    return 1

def scissors_agent(observation, configuration):
    return 2

def smart_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)
    

In [5]:

def cycle_agent(observation, configuration):
    """Агент чередует ходы по циклу: Камень → Бумага → Ножницы"""
    return observation.step % configuration.signs

def losing_agent(observation, configuration):
    """Агент всегда выбирает действие, которое проигрывает предыдущему ходу противника"""
    if observation.step > 0:
        return (observation.lastOpponentAction - 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

def smart_random_agent(observation, configuration):
    """Агент с 70% вероятностью повторяет успешный ход, иначе выбирает случайно"""
    if observation.step > 0 and random.random() < 0.7:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

def bias_rock_agent(observation, configuration):
    """Агент с 60% вероятностью выбирает Камень"""
    return 0 if random.random() < 0.6 else random.randrange(1, configuration.signs)

def bias_paper_agent(observation, configuration):
    """Агент с 60% вероятностью выбирает Бумагу"""
    return 1 if random.random() < 0.6 else random.randrange(0, configuration.signs)

def bias_scissors_agent(observation, configuration):
    """Агент с 60% вероятностью выбирает Ножницы"""
    return 2 if random.random() < 0.6 else random.randrange(0, configuration.signs)
    

## Запуск турнира между агентами

In [10]:

# Инициализация среды
env = make("rps", configuration={"episodeSteps": 100})

# Список агентов
agents = [
    rock_agent, copy_opponent, random_agent, paper_agent, scissors_agent, smart_agent,
    cycle_agent, losing_agent, smart_random_agent, bias_rock_agent,
    bias_paper_agent, bias_scissors_agent
]

# Турнир
results = []
for i, agent1 in enumerate(agents):
    row = []
    for j, agent2 in enumerate(agents):
        env.reset()
        env.run([agent1, agent2])  # Запуск игры между двумя агентами
        rewards = env.state
        row.append(rewards[0]['reward'])  # Сохраняем результат для первого агента
    results.append(row)

# Форматирование результатов в таблицу
columns = [
    "Rock Agent", "Copy Opponent", "Random Agent", "Paper Agent", 
    "Scissors Agent", "Smart Agent", "Cycle Agent", "Losing Agent", 
    "Smart Random Agent", "Bias Rock Agent", "Bias Paper Agent", 
    "Bias Scissors Agent"
]
results_df = pd.DataFrame(results, columns=columns, index=columns)

# Вывод таблицы
print("Результаты турнира:")
print(results_df)
    

Результаты турнира:
                     Rock Agent  Copy Opponent  Random Agent  Paper Agent  \
Rock Agent                  0.0            0.0             0        -99.0   
Copy Opponent               0.0            0.0             0          0.0   
Random Agent                0.0            0.0             0          0.0   
Paper Agent                99.0            0.0             0          0.0   
Scissors Agent            -99.0            0.0             0         99.0   
Smart Agent                99.0           50.0             0         98.0   
Cycle Agent                 0.0           98.0             0          0.0   
Losing Agent              -97.0           99.0             0        -99.0   
Smart Random Agent         63.0            0.0             0         65.0   
Bias Rock Agent             0.0            0.0             0        -43.0   
Bias Paper Agent           66.0            0.0             0          0.0   
Bias Scissors Agent       -65.0            0.0          